## Hi this notebook is for a Coursera assigment ^_^

In [1]:
import numpy as np
import pandas as pd
import folium 
import json # library to handle JSON files
import requests # library to handle requests


In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Now I'll start week three projct 
### Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [3]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df.head(12)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [4]:
df.groupby('Borough').nunique()

,Postcode,Borough,Neighbourhood
Borough,,,
Central Toronto,9,1,17
Downtown Toronto,18,1,36
East Toronto,5,1,7
East York,5,1,6
Etobicoke,12,1,45
Mississauga,1,1,1
North York,24,1,38
Not assigned,77,1,1
Queen's Park,1,1,1


In [5]:
# remove any Not assigned Borough, 
df2 = df[df['Borough'] != "Not assigned"].reset_index(drop=True)
# group all Borough with the same Postcode
df3 = df2.groupby(["Postcode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
# if Neighbourhood is Not assigned then make it = to Borough
for index, row in df3.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
# show the number of rows,culms
df3.shape 

(103, 3)

In [6]:
# read the coordinates from the csv file on Coursera
geoCoord = pd.read_csv("http://cocl.us/Geospatial_data")
# rename the column "Postcode"
geoCoord.rename(columns={"Postal Code": "Postcode"}, inplace=True)
# merge the two tables
df4 = df3.merge(geoCoord, on="Postcode", how="left")
df4.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## now we are ready to work ^_^ 
### first I'll show the mape

In [7]:
latitude =  43.653963
longitude = -79.387207
map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)  
    
map

### now I'll use th API  I have a part with the CLIENT_ID and CLIENT_SECRET but I'll deleate it  becuse it should not be public 

In [9]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Postcode'], df4['Borough'], df4['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
# now venues array has all of the venues from the api

In [10]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Postcode', 'Borough', 'Neighbourhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2255, 9)


,Postcode,Borough,Neighbourhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


### we can count how many venus are in a neighbourhood 
#### it donse't look nice but we can get the data we need 

In [11]:
venues_df.groupby(["Postcode", "Borough", "Neighbourhood"]).count() 

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Postcode,Borough,Neighbourhood,,,,,,
M1B,Scarborough,"Rouge, Malvern",1,1,1,1,1,1
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",2,2,2,2,2,2
M1E,Scarborough,"Guildwood, Morningside, West Hill",7,7,7,7,7,7
M1G,Scarborough,Woburn,3,3,3,3,3,3
M1H,Scarborough,Cedarbrae,7,7,7,7,7,7
M1J,Scarborough,Scarborough Village,1,1,1,1,1,1
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",4,4,4,4,4,4
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",3,3,3,3,3,3


In [12]:
print('{} venues were returned by Foursquare.'.format(venues_df.shape[0]))

2255 venues were returned by Foursquare.


In [13]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 270 uniques categories.
